In [118]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

In [177]:
save_dir = '/home/jl277509/Documents/Ablation_2_models_full/rotation_translation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/rotation_translation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/rotation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/rotation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/translation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/7_translation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/cutin'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/10_cutin_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/cutin'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/11_cutout_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimdepth_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/8_trimdepth_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimextremities_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/9_trimextremities_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutin_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/10_cutin_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutout_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/6_no_augm.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'
#n_models = 2

#save_dir = '/home/jl277509/Documents/Ablation_2_models/rotation_translation.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/rotation_translation'
#n_models = 1


# Step 1 : recover all perf values (with cross val std)

In [178]:
sub_dirs = {'SOr': [f'{basedir}/SOr_left_UKB40/', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'cv_score', 'cv_std', 'splits_score'], # NB : error in pipeline stds -> std ... ## ONLY IN CUTOUT : rerun
            'FIP': [f'{basedir}/FIP_right_UKB40/', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'cv_score', 'cv_std', 'splits_score'],
            'SC-sylv': [f'{basedir}/SC-sylv_left_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_central_left_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_1': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_cingulate_right_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_2': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', ['ACC_custom_embeddings/Right_PCS/test_values.json'], 'cv_score', 'cv_std', 'splits_score']
            }

l = os.listdir(basedir)
root_names = [elem for elem in l if ('UKB40' not in elem) and '#' not in elem]

In [179]:
# store everything globally
l_model_glob = []
l_perf_glob = []
l_std_glob = []
l_splits_scores_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, score, std, split_score) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            l_stds = []
            l_splits_scores = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...

                if isinstance(perf[score], float):
                    value = perf[score]
                else:
                    value = perf[score][0]
                l_perfs.append(value)
                if isinstance(perf[std], float):
                    value = perf[std]
                else:
                    value = perf[std][0]
                l_stds.append(value)
                value = perf[split_score] # is a list
                l_splits_scores.append(value)
                
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_std_glob.append(l_stds)
            l_splits_scores_glob.append(l_splits_scores)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [180]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score', 'std']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob), flatten_sum(l_std_glob)]).T
df = pd.DataFrame(data=values, columns=cols)
df['splits_score']=flatten_sum(l_splits_scores_glob)

# Step 2 : reorganize the df with the parameters

Rotation - Translation

In [8]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_0' in model:
        list_magnitude_trans.append(0)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_6' in model:
        list_magnitude_trans.append(6)

    if 'rot_6_' in model:
        list_magnitude_rot.append(6)
    elif 'rot_12' in model:
        list_magnitude_rot.append(12)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Rotation - Translation : second round

In [123]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_0' in model:
        list_magnitude_trans.append(0)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_6' in model:
        list_magnitude_trans.append(6)

    if 'rot_6_' in model:
        list_magnitude_rot.append(6)
    elif 'rot_12' in model:
        list_magnitude_rot.append(12)
    elif 'rot_18' in model:
        list_magnitude_rot.append(18)
    elif 'rot_24' in model:
        list_magnitude_rot.append(24)
    elif 'rot_30' in model:
        list_magnitude_rot.append(30)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Rotation only

In [142]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'rot_6_' in model:
        list_magnitude.append(6)
    elif 'rot_12' in model:
        list_magnitude.append(12)
    elif 'rot_18' in model:
        list_magnitude.append(18)
    elif 'rot_24' in model:
        list_magnitude.append(24)
    elif 'rot_30' in model:
        list_magnitude.append(30)
    elif 'rot_45' in model:
        list_magnitude.append(45)
    elif 'rot_60' in model:
        list_magnitude.append(60)
    elif 'rot_90' in model:
        list_magnitude.append(90)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Translation

In [156]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'translation_1_' in model:
        list_magnitude.append(1)
    elif 'translation_2' in model:
        list_magnitude.append(2)
    elif 'translation_3' in model:
        list_magnitude.append(3)
    elif 'translation_4' in model:
        list_magnitude.append(4)
    elif 'translation_5' in model:
        list_magnitude.append(5)
    elif 'translation_6' in model:
        list_magnitude.append(6)
    elif 'translation_7' in model:
        list_magnitude.append(7)
    elif 'translation_8' in model:
        list_magnitude.append(8)
    elif 'translation_9' in model:
        list_magnitude.append(9)
    elif 'translation_10' in model:
        list_magnitude.append(10)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Trimdepth

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_mode=[]
list_proba=[]
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    elif '6mm' in model:
        list_magnitude.append(6)
    elif '8mm' in model:
        list_magnitude.append(8)

    if 'constant' in model:
        list_mode.append('constant')
    else:
        list_mode.append('per branch')

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Magnitude']=list_magnitude
df['Mode']=list_mode
df['Proba'] = list_proba
parameters=['Magnitude', 'Mode', 'Proba']

Trimextremities

In [31]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'pepper50' in model:
        list_magnitude.append(50)
    elif 'pepper100' in model:
        list_magnitude.append(100)

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

cutin

In [169]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '5_15' in model:
        list_magnitude.append('[05,15]')
    elif '10_30' in model:
        list_magnitude.append('[10,30]')
    elif '20_40' in model:
        list_magnitude.append('[20,40]')
    elif '30_50' in model:
        list_magnitude.append('[30,50]')
    elif '40_60' in model:
        list_magnitude.append('[40,60]')
    elif '50_70' in model:
        list_magnitude.append('[50,70]')
    elif '70_90' in model:
        list_magnitude.append('[70,90]')
    elif '20_70' in model:
        list_magnitude.append('[20,70]')
    elif '20_90' in model:
        list_magnitude.append('[20,90]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

cutout

In [181]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '01_5' in model:
        list_magnitude.append('[01,5]')
    elif '5_15' in model:
        list_magnitude.append('[05,15]')
    elif '10_20' in model:
        list_magnitude.append('[10,20]')
    elif '15_25' in model:
        list_magnitude.append('[15,25]')
    elif '20_30' in model:
        list_magnitude.append('[20,30]')
    elif '30_40' in model:
        list_magnitude.append('[30,40]')
    elif '40_50' in model:
        list_magnitude.append('[40,50]')
    elif '50_60' in model:
        list_magnitude.append('[50,60]')
    elif '60_70' in model:
        list_magnitude.append('[60,70]')
    elif '70_80' in model:
        list_magnitude.append('[70,80]')
    elif '5_30' in model:
        list_magnitude.append('[05,30]')
    elif '5_40' in model:
        list_magnitude.append('[05,40]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

# Create tables

In [182]:
df_reformat = df.copy()
df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
df_reformat = df_reformat.drop(labels='model', axis='columns')
df_reformat['training_number'] = df_reformat.groupby(parameters+['task', 'region']).cumcount()
print('Check number of models for each config')
print(np.unique(df_reformat.groupby(parameters+['task', 'region']).cumcount(), return_counts=True))
## remove rows when more than 2 models
#df_reformat = df_reformat.drop(index=df_reformat.loc[df_reformat['training_number']>1].index.tolist(), axis='row')
parameters.append('training_number')

Check number of models for each config
(array([0, 1]), array([420, 414]))


In [183]:
#df_reformat = df.copy()
#df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
#df_reformat = df_reformat.drop(labels='model', axis='columns')
#df_reformat['training_number']=[k%n_models for k in range(len(df_reformat))]
#parameters.append('training_number')

In [184]:
# Pivoting the table
df_pivot = df_reformat.pivot(index=parameters, columns=['task', 'region'], values=['score', 'std', 'splits_score'])
df_pivot = df_pivot.dropna()

# Renaming columns
df_pivot.columns = [f'score_{col}' for col in df_pivot.columns]

# Resetting index to get a flat DataFrame
df_pivot = df_pivot.reset_index()

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("SC-sylv" in elem)]
df_pivot['score_Isomap_SC-sylv_left_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("Cing" in elem)]
df_pivot['score_Isomap_Cing_right_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

In [185]:
df_scores = df_pivot[["score_('score', 'Left_OFC', 'SOr')", "score_('score', 'Right_FIP', 'FIP')", "score_('score', 'Right_PCS', 'Cing_2')",
                      "score_Isomap_SC-sylv_left_global", "score_Isomap_Cing_right_global"]].apply(pd.to_numeric).mul(100).round(1)
df_scores.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# get stds for Isomap
# SC-sylv
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_central_left_dim{k}', 'SC-sylv')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_sc_sylv = avg_per_split.apply(lambda row : np.std(row.tolist()))
# Cing
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_cingulate_right_dim{k}', 'Cing_1')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_cing = avg_per_split.apply(lambda row : np.std(row.tolist()))

# get the std df
df_stds = df_pivot[["score_('std', 'Left_OFC', 'SOr')", "score_('std', 'Right_FIP', 'FIP')", "score_('std', 'Right_PCS', 'Cing_2')"]]
df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
df_stds['isomap_Cing_right']=std_isomap_cing
df_stds = df_stds.apply(pd.to_numeric).mul(100).round(1)
df_stds.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# combine in one df
formatted_df = df_scores.astype(str) + "(" + df_stds.astype(str) + ")"

formatted_df = pd.concat((df_pivot[parameters], formatted_df), axis=1)

/tmp/ipykernel_599240/1976764873.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
/tmp/ipykernel_599240/1976764873.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_Cing_right']=std_isomap_cing


In [186]:
from scipy.stats import rankdata

# just use scores, no std
df_scores_only = pd.concat((df_pivot[parameters[:-1]], df_scores), axis=1)

# Compute mean metric per method
df_mean = df_scores_only.groupby(parameters[:-1]).mean().round(1)
# Rank each metric (lower values are better)
ranks = df_mean.apply(lambda x: rankdata(-x, method='average'), axis=0)

# Compute average rank
df_mean["Avg. Rank"] = ranks.mean(axis=1).round(1)

In [187]:
formatted_df

,Magnitude,Bottom_Proba,Proba,training_number,SOr,FIP,PCS,Isomap_SC-sylv_left,Isomap_Cing_right
0,"[01,5]",50,40,0,63.8(3.0),61.0(9.1),62.8(7.9),28.8(2.0),4.4(1.0)
1,"[01,5]",50,40,1,64.2(1.2),66.1(8.9),62.5(8.6),35.8(3.5),4.6(0.6)
2,"[01,5]",50,80,0,67.9(2.5),64.0(2.9),63.2(6.2),30.5(3.5),6.1(1.4)
3,"[01,5]",50,80,1,64.3(3.7),67.7(6.0),64.0(6.5),37.3(4.0),2.8(1.4)
4,"[05,15]",50,40,0,72.3(2.5),70.6(6.0),80.9(5.3),54.2(4.3),9.9(1.8)
5,"[05,15]",50,40,1,74.4(2.5),75.7(3.5),76.9(5.5),55.3(3.6),8.4(2.0)
6,"[05,15]",50,80,0,73.6(2.3),80.6(4.4),83.8(6.0),55.0(4.1),15.3(2.4)
7,"[05,15]",50,80,1,71.4(2.4),79.2(5.6),86.7(5.7),56.3(3.2),14.9(1.7)
8,"[05,15]",70,40,0,68.8(2.9),71.7(7.3),66.1(3.1),43.2(3.6),7.1(1.1)
9,"[05,15]",70,40,1,67.7(2.0),74.3(4.3),70.4(4.3),55.2(3.9),8.0(1.4)


In [188]:
df_mean

SOr   FIP   PCS  Isomap_SC-sylv_left  \
Magnitude Bottom_Proba Proba                                          
[01,5]    50           40     64.0  63.6  62.6                 32.3   
                       80     66.1  65.8  63.6                 33.9   
[05,15]   50           40     73.4  73.2  78.9                 54.8   
                       80     72.5  79.9  85.2                 55.6   
          70           40     68.2  73.0  68.2                 49.2   
                       80     71.9  77.6  81.4                 56.0   
[05,30]   50           80     67.7  80.2  85.0                 56.8   
[05,40]   50           80     61.8  79.8  84.5                 57.0   
[10,20]   50           40     72.6  73.6  78.2                 56.0   
                       80     66.6  78.0  84.4                 57.8   
          70           40     71.0  74.3  79.2                 47.8   
                       80     72.7  78.4  83.9                 55.8   
[15,25]   50           40     72.4  78.9  78.3                 54.5   
                       80     63.2  80.8  86.0                 57.7   
          70           40     71.6  78.8  82.1                 56.0   
                       80     66.4  79.1  87.4                 56.3   
[20,30]   50           40     72.3  80.2  85.0                 56.3   
                       80     63.2  78.7  86.2                 57.4   
          70           40     70.8  78.4  80.5                 56.0   
                       80     67.6  79.8  87.2                 55.4   
[30,40]   50           40     66.6  79.7  85.6                 57.0   
                       80     61.2  79.6  87.0                 57.4   
[40,50]   50           40     66.1  79.5  84.6                 55.2   
                       80     58.9  78.4  86.2                 35.8   
[50,60]   50           40     58.0  78.8  85.0                 55.2   
                       80     58.8  77.5  85.4                 55.6   
[60,70]   50           80     55.0  76.2  84.6                 54.6   
[70,80]   50           80     55.5  64.9  76.3                 51.6   

                              Isomap_Cing_right  Avg. Rank  
Magnitude Bottom_Proba Proba                                
[01,5]    50           40                   4.5       26.0  
                       80                   4.4       25.1  
[05,15]   50           40                   9.2       18.4  
                       80                  15.1        8.6  
          70           40                   7.6       22.4  
                       80                  10.7       15.7  
[05,30]   50           80                  17.4        7.3  
[05,40]   50           80                  17.0       10.6  
[10,20]   50           40                  11.5       16.1  
                       80                  16.3       11.6  
          70           40                  10.9       19.6  
                       80                  12.6       13.4  
[15,25]   50           40                   9.8       16.9  
                       80                  16.8        7.3  
          70           40                  12.8       13.4  
                       80                  15.1        9.2  
[20,30]   50           40                  13.5        8.8  
                       80                  18.2        8.9  
          70           40                   9.8       16.2  
                       80                  16.3        9.2  
[30,40]   50           40                  14.5        9.6  
                       80                  16.9        8.7  
[40,50]   50           40                  14.9       14.1  
                       80                  18.4       14.3  
[50,60]   50           40                  14.4       16.6  
                       80                  17.7       14.3  
[60,70]   50           80                  14.6       19.3  
[70,80]   50           80                  11.3       24.4

# Save df

In [163]:
formatted_df.to_csv(save_dir+'.csv', index=False)

In [164]:
df_mean.to_csv(save_dir+'rank.csv', index=False)